# Modelo VaR para un portafolio de bonos

**Descripción general**
-------------------
Este algoritmo reproduce la estimación de de sensibilidades (Duración, Convexidad) y el VaR para un portafolio de bonos del Tesoro de EE.UU.("US Treasury Bonds") usando Python. El algoritmo permite estimar el VaR Paramétrico/histórico y con volatilidad EWMA y realizar pruebas de backtesting (por ej. Kupiec).

-----------------------

1. Conexión y descarga de datos:
   - Se conecta y descarga desde FRED las tasas de bonos del Tesoro a diferentes vencimientos (DGS1MO, DGS3MO, DGS6MO, DGS1, DGS2, DGS3, DGS5, DGS7, DGS10, DGS20, DGS30).
   - Limpia los datos.

2. Construcción de la curva cero cupón: se usa Bootstrap en Python interpolando tasas para apróximar la curva.

   Extensiones del algortimo:
    - usar modelos de Nelson & Siegel o Svensson
    - usar QuantLib para bootstrap de curva cero cupón u otra librería alternativa.

3. Creación y valoración del portafolio:
   - Define un portafolio de bonos (maturity, cupón, nominal) a distintos plazos.
   - Valora diariamente los bonos con la curva cero cupón obtenida.
   - Calcula el valor total del portafolio.

4. Estimación del VaR histórico:
   - Genera escenarios aplicando shocks.
   - Calcula la distribución de pérdidas y ganancias (P&L).
   - Obtiene VaR al 99%.

5. Estimación del VaR con volatilidad EWMA:
   - Ajusta los retornos históricos por volatilidad condicional (EWMA) para reflejar la volatilidad actual del mercado.

   Extensiones del algortimo: usar modelos tipo GARCH(1,1) u otras extensiones.

6. Backtesting:
   - Evalúa excepciones (pérdidas reales que exceden el VaR).
   - Aplica pruebas estadísticas de Kupiec (proporción de fallos).



Entradas del algoritmo:
--------
- Series históricas de rendimientos/“par yields” del Tesoro a distintos plazos.
- Definición del portafolio de bonos (vencimiento, cupón, nominal).
- Parámetros de ventana histórica, nivel de confianza y lambda EWMA.

Salidas:
-------
- Serie temporal de valores diarios del portafolio.
- Serie de P&L obervados.
- VaR histórico y EWMA al 99%.
- Resultados de backtesting (estadísticos y p-valores).
- Gráficos opcionales de VaR vs P&L.

Dependencias
------------
Obligatorias:
    *numpy pandas matplotlib scipy pandas_datareader*

Notas adicionales:
-----
- El portafolio puede ampliarse fácilmente a más bonos especificando sus características.


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.interpolate import interp1d             # Reemplaza np.interp()
from pandas_datareader import data as web
import matplotlib.pyplot as plt
# Si se rqueren ajustes en los formatos de fechas
#import datetime
#from datetime import timedelta
#import matplotlib.dates as mdates
import warnings  # Manejo de warnings en las salidas del codigo
warnings.filterwarnings('ignore')

In [ ]:
# !pip install --q QuantLib-Python
# import QuantLib as ql

Yields (curva) y bonos
---
Yields por nodos (DGS1MO, DGS3MO,..., DGS30) son cotizaciones de mercado que dan la forma de la curva de tipos de interés (i.e. una tasa por cada vencimiento estándar). Esa curva te permite construir una curva cero cupón (descuento) y, con ella, valorar cualquier bono qe negocie en el mercado y a cualquier vencimiento.

In [ ]:
#from pandas_datareader import data as web  # módulo adecuado para usar FRED
#----------------------------------------------------------------------------
# Descargar y limpiar rendimientos (FRED) → df_par (DataFrame, T x M).
!pip install --q fredapi
from fredapi import Fred
fred = Fred(api_key='d3d779c3fb14e77f95d792940ab24895')   # Cambiar el codigo de la API de su cuenta personal

In [ ]:
start = '2020-01-01'    # Ajustar las fechas segun la aplicacion a desarrollar
end = '2024-12-31'

In [ ]:
## Como funciona "FRED data" (de forma directa)
#fred.search('DGS1MO')  # Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity / #data[start:end]

In [ ]:
# Lista de series FRED (Treasury Constant Maturity)
bonds = ['DGS1MO','DGS3MO','DGS6MO','DGS1','DGS2','DGS3','DGS5','DGS7','DGS10','DGS20','DGS30']

# Descarga las series en un DataFrame
bonds_yield = pd.DataFrame()

for bond in bonds:
    series = web.DataReader(bond, 'fred')  # descarga cada serie
    bonds_yield[bond] = series[bond]       # agrega columna con ese nombre

# Limpiar datos faltantes:
bonds_yield = bonds_yield.dropna()  # Los datos se limpian eliminando filas con valores nulos (.dropna())
bonds_yield = bonds_yield / 100     # Las yields se convierten de porcentaje a decimal (dividiendo por 100).

**Nota para el manejo de datos faltantes**: En lugar de usar 'dropna()', que elimina días (o fechas enteras) si faltan muy pocos datos, podrías usar métodos de interpolación (.interpolate() o .fillna(0)) para rellenar valores faltantes, lo cual es común en series de tiempo financieras.

In [ ]:
## Verificar info
bonds_yield.head(3)

,DGS1MO,DGS3MO,DGS6MO,DGS1,DGS2,DGS3,DGS5,DGS7,DGS10,DGS20,DGS30
DATE,,,,,,,,,,,
2020-10-05,0.0009,0.001,0.0011,0.0012,0.0014,0.0019,0.0033,0.0055,0.0078,0.0134,0.0157
2020-10-06,0.0008,0.001,0.0011,0.0014,0.0014,0.0017,0.0032,0.0053,0.0076,0.0133,0.0156
2020-10-07,0.0008,0.001,0.0012,0.0013,0.0016,0.0021,0.0035,0.0056,0.0081,0.0137,0.0160


In [ ]:
plt.figure(figsize=(11,3))
plt.plot(bonds_yield.index,bonds_yield, label=bonds_yield.columns)
plt.title('Yields (Tasas) en cada fecha ')
plt.ylabel('Yields')
plt.legend();

Portafolio de bonos
---
* Instrumento A: 2 años, cupón 2.5%, notional 1,000,000
* Instrumento B: 5 años, cupón 3.0%, notional 2,000,000
* Instrumento C: 10 años, cupón 3.5%, notional 1,000,000


In [ ]:
bonos = [
    {'Bono': '2y', 'nocional': 1_000_000, 'cupon': 0.025, 'vencimiento': 2, 'frec_cupon': 2},
    {'Bono': '5y', 'nocional': 2_000_000, 'cupon': 0.030, 'vencimiento': 5, 'frec_cupon': 2},
    {'Bono': '10y', 'nocional': 1_500_000, 'cupon': 0.035, 'vencimiento': 10, 'frec_cupon': 2}
]
pd.DataFrame(bonos)

,Bono,nocional,cupon,vencimiento,frec_cupon
0,2y,1000000,0.025,2,2
1,5y,2000000,0.030,5,2
2,10y,1500000,0.035,10,2


# 2.	Convertir par yields Curva Cero Cupón (por fecha) → zero_curve (T x M)

Los rendimientos de FRED forman una curva de rendimientos al par (par yield curve). Para valorar correctamente un bono con cupones, necesitamos descontar cada uno de sus flujos de caja futuros (cupones y principal) con la tasa de interés "cero cupón" correspondiente a la fecha de cada flujo.

El bootstrapping es una de las metodologías que se utilizan para cosntruir la curva cero cupón a partir de la curva al par.

**Bootstrap → curva cero cupón**

A partir de los par yields se obtiene la tasa cero cupón para cada nodo (tenor).

**Metodología:**

Vencimientos Cortos (T < 1 año): Para tenores $T < 1$ año (ej. 1m, 3m, 6m) se apróxima como instrumento de descuento (Se asume que los instrumentos son de descuento puro).

$$ DF = \frac{1}{(1 + r)^T} ≈ e^{-r \times T} ⇒ z = \frac{−ln(DF)}{T} $$

donde $z$ es la tasa cero en capitalización continua. La tasa cero ($z$) se deriva de la tasa al par (con capitalización simple, $r_{par}$).


Vencimientos Largos (T ≥ 1 año): Para tenores $T \ge 1$ año con cupones semestrales (freq=2). Se utiliza un proceso iterativo. Si un bono al par vale 100, se despeja el factor de descuento ($DF_m$) del último flujo de caja, utilizando los factores de descuento ya calculados para los cupones anteriores. Si un $DF$ intermedio no se conoce, se interpola linealmente a partir de las tasas cero ya calculadas.

Supongamos $m$ pagos $m = T \times \text{freq}$, cupón por periodo $c = r_{par} / \text{freq}$.

Valor presente de los cupones anteriores (con DFs ya calculados o interpolados):

$$ PV_{known} = \sum_{𝑘=1}^{𝑚−1} c \times 100 \times DF(t_k)$$

Resolver $DF_m$ usando que bono a la par vale 100. Una vez despejado $DF_m$, la tasa cero se calcula como:

$$ z= -ln(DF_m)/T_m $$

**Supuestos y limitaciones**

Interpolación de tasas para fechas intermedias usa interpolación de tasas continuas (se interpola $-ln(DF)/t#).

Se puede hacer una mejora de esta aplicaicón usando librerías como QuantLib, que manejan convenciones de conteo de días, fechas de liquidación y métodos de interpolación más sofisticados (ej. log-linear, cúbica), produciendo curvas idénticas a las del mercado.

### Funciones

In [ ]:
def bootstrap_zero_curve(par_rates, tenors_months, coupon_freq=2):
    """Convierte una curva de rendimientos al par a una curva cero cupón continua."""
    tenors_years = np.array(tenors_months, dtype=float) / 12
    zero_rates = np.zeros(len(par_rates))
    dfs_by_time = {}

    for i, (T, r_par) in enumerate(zip(tenors_years, par_rates)):
        if T <= 0: continue
        if T <= 1:
            df = 1 / (1 + r_par * T)
            zero_rates[i] = -np.log(df) / T if df > 0 else 0
        else:
            m = int(round(T * coupon_freq))
            coupon = r_par / coupon_freq
            pv_known_coupons = 0

            # Interpolar DFs para cupones intermedios
            known_times = np.array(sorted(dfs_by_time.keys()))
            if len(known_times) > 0:
                known_zeros = np.array([-np.log(dfs_by_time[t]) / t for t in known_times])
                interp_func = interp1d(known_times, known_zeros, kind='linear', fill_value='extrapolate') # np.interp()

            for k in range(1, m):
                t_k = k / coupon_freq
                if round(t_k, 8) in dfs_by_time:
                    df_k = dfs_by_time[round(t_k, 8)]
                elif len(known_times) > 0:
                    r_interp = float(interp_func(t_k))
                    df_k = np.exp(-r_interp * t_k)
                else: # Para el primer bono largo plazo
                    df_k = 1 / (1 + par_rates[i] * t_k)
                pv_known_coupons += coupon * df_k

            df_m = (1 - pv_known_coupons) / (1 + coupon)
            zero_rates[i] = -np.log(df_m) / T if df_m > 0 else (zero_rates[i-1] if i > 0 else 0)

        dfs_by_time[round(T, 8)] = np.exp(-zero_rates[i] * T)

    return zero_rates

In [ ]:
# Mapeo de columnas a tenores (o nodos) en meses
months = [1, 3, 6, 12, 24, 36, 60, 84, 120, 240, 360]   # T < 12 (menor a un año) | T > 12 (mayor a un año)

zero_curves_hist = np.array([bootstrap_zero_curve(row, months) for index, row in bonds_yield.iterrows()])
                                                               # df.iterrows(): (Pandas) se utiliza para iterar sobre las filas de un DataFrame
tenors_years = np.array(months) / 12

In [ ]:
plt.figure(figsize=(9,3))
plt.plot(bonds_yield.index,zero_curves_hist, label=months)
plt.title('Tasas cero cupón en cada t')
plt.ylabel('Tasas cero cupón')
plt.legend();

In [ ]:
# Un ejemplo: para un fecha t
par_row = bonds_yield.iloc[-1].values
zero_curve_t1 = bootstrap_zero_curve(par_row, months)
print('zero curve (first date):', zero_curve_t1.round(4)*100)

zero curve (first date): [4.16 3.99 3.76 3.56 3.52 3.53 3.66 3.87 4.13 4.84 4.71]


**Otra mejora:** En lugar del bootstrap, puedes modelar la curva con funciones paramétricas como Nelson-Siegel o su extensión Svensson. Estos modelos son excelentes para suavizar la curva y obtener tasas para cualquier vencimiento, incluso aquellos sin datos de mercado directos.

# Valoración (Pricing) de los bonos

El valor de un portafolio de bonos es la suma de los valores individuales de cada bono. El valor (precio) de un bono individual se calcula como el valor presente de todos sus flujos de caja futuros, descontados con las tasas de la curva cero cupón.

* Se generan cashflows y tiempos (en años) para cada bono: cupones periódicos + principal.

$$ P = \sum_{i} CF_i\ e^{-z_i \times t_i } $$

Donde $CF_i$ es el flujo de caja en el tiempo $t_i$ y $z_i$ es la tasa cero para ese tiempo.

En el script se valora cada bono del portafolio y se suma en portfolio_values[i].

Nota: la implementación usa años y frecuencia fija; no implementa convenciones de day-count (ej., 30/360 o ACT/365).

In [ ]:
def get_portfolio_cashflows(portfolio):
    """Pre-calcula los flujos de caja para toda la cartera."""
    portfolio_cfs = []
    for bono in portfolio:
        num_cupones = int(bono['vencimiento'] * bono['frec_cupon'])
        monto_cupon = (bono['cupon'] / bono['frec_cupon']) * bono['nocional']
        tiempos = np.arange(1, num_cupones + 1) / bono['frec_cupon']
        flujos = np.full(num_cupones, monto_cupon)
        flujos[-1] += bono['nocional']
        portfolio_cfs.append({'tiempos': tiempos, 'flujos': flujos})
    return portfolio_cfs

def valor_presente(portfolio_cfs, curve_zero, tenors_years):
    """Función de valoración que usa flujos de caja pre-calculados."""
    total_pv = 0
    interp_func = interp1d(tenors_years, curve_zero, kind='linear', fill_value='extrapolate')

    for bono_cfs in portfolio_cfs:
        tasas_cero = interp_func(bono_cfs['tiempos'])
        factores_descuento = np.exp(-tasas_cero * bono_cfs['tiempos'])
        total_pv += np.sum(bono_cfs['flujos'] * factores_descuento)
    return total_pv

def portfolio_pv_and_sensitivities(portfolio_cfs, curve_zero, tenors_years, shock=0.0001):
    pv_base = valor_presente(portfolio_cfs, curve_zero, tenors_years)

    pv_up = valor_presente(portfolio_cfs, curve_zero + shock, tenors_years)
    pv_down = valor_presente(portfolio_cfs, curve_zero - shock, tenors_years)

    duration = (pv_down - pv_up) / (2 * pv_base * shock) if pv_base else 0
    convexity = (pv_down + pv_up - 2 * pv_base) / (pv_base * shock**2) if pv_base else 0

    return pv_base, duration, convexity

In [ ]:
portfolio_cfs = get_portfolio_cashflows(bonos)
portfolio_pv, portfolio_duration, portfolio_convexity = portfolio_pv_and_sensitivities(portfolio_cfs,
                                                                                       zero_curve_t1, tenors_years)


In [ ]:
# Valor del portafolio y sensibilidades
print("\n VALOR Y SENSIBILIDADES")
print(f"   - Valor Presente del Portafolio: ${portfolio_pv:,.2f}")
print(f"   - Duración Modificada: {portfolio_duration:.4f}")

Estimación del VaR Histórico y P&L diario
---
El Value at Risk (VaR) es una medida de riesgo que estima la pérdida máxima potencial de un portafolio en un horizonte de tiempo determinado (ej. 1 día) con un nivel de confianza específico (ej. 95% o 99%).
El método de VaR Histórica consiste en:

* Observar los cambios históricos (shocks) en los factores de riesgo (en este caso, las tasas de la curva cero).

* Aplicar cada uno de esos shocks históricos al estado actual de los factores de riesgo para generar una serie de escenarios futuros hipotéticos.

* Revalorar el portafolio en cada uno de estos escenarios.

* Construir una distribución de pérdidas y ganancias (P&L) y encontrar el percentil correspondiente al nivel de confianza.


**Implementación**

* Se calculan los cambios diarios históricos (retornos) de todas las tasas de la curva cero: zero_returns = returns_matrix(zero_curves)

* Se itera a través de la "ventana de prueba" (test window). Para cada día t:

* Se toma la curva cero del día anterior (t-1): prev_tenors = zero_curves[t-1, :].

* Se aplican los últimos 250 retornos históricos (lookback) a esa curva para generar 250 escenarios de curvas futuras (aquí es donde se debe aplicar la corrección a retornos logarítmicos).

* Se revalora el portafolio en cada uno de los 250 escenarios: zeros_s → obtenemos PortValues_scen[s].

* Se calcula el P&L simulado para cada escenario (Valor del escenario - Valor en t-1): SimulatedPandL[s] = PortValues_scen[s] - portfolio_values[t-1].

* La función historical_var ordena las pérdidas y extrae el percentil 95 y 99 para obtener el VaR: devuelve el percentil de la distribución SimulatedPandL.

**Nota**: sobre retornos vs log-returns: aplicar retornos relativos (1 + r) puede producir curvas negativas si las tasas son pequeñas y cambios negativos muy grandes.

In [ ]:
def calculate_analytical_var(precio, duracion, volatilidad, z_alpha):
    return precio * duracion * (volatilidad * z_alpha)

def kupiec_pof_test(failures, n_obs, alpha):
    if n_obs == 0: return 1.0, 1.0
    failure_rate = np.clip(failures / n_obs, 1e-9, 1 - 1e-9)
    log_l0 = failures * np.log(failure_rate) + (n_obs - failures) * np.log(1 - failure_rate)
    log_l1 = failures * np.log(alpha) + (n_obs - failures) * np.log(1 - alpha)
    lr_stat = -2 * (log_l1 - log_l0)
    return 1 - stats.chi2.cdf(lr_stat, 1)

In [ ]:
# C. Parámetros de Riesgo
alpha = 0.99
z_alpha = stats.norm.ppf(alpha)
T = 250
lambd = 0.94
volatilidad = 0.0008

var_param = calculate_analytical_var(portfolio_pv, portfolio_duration, volatilidad, z_alpha)

print("\n VaR Paramétrico")
print(f"   - VaR a 1 día al {alpha*100:.0f}%: ${var_param:,.2f}")

Backtesting
---
El backtesting es el proceso de validar la precisión de un modelo de VaR comparando sus predicciones con las pérdidas y ganancias reales observadas.

**Prueba de Kupiec (Proporción de Fallos)**: Evalúa si el número de excepciones (días en que la pérdida real excedió el VaR) es consistente con el nivel de confianza. Por ejemplo, para un VaR al 99%, se espera una excepción cada 100 días.

**Implementación**

* Se identifican los días en que la pérdida real fue mayor que el VaR predicho.

* Se implementan las pruebas de ratio de verosimilitud (likelihood ratio) para Kupiec, calculando el estadísticos LR y su p-valor a partir de una distribución Chi-cuadrado.

* Un p-valor alto (generalmente > 0.05) indica que no se puede rechazar la hipótesis nula de que el modelo es correcto.

Estimación del VaR con Volatilidad EWMA (Simulación Histórica Filtrada)
---

La simulación histórica simple asume que los retornos pasados son igualmente probables en el futuro. Esto puede ser problemático si la volatilidad del mercado ha cambiado recientemente. La Simulación Histórica ajusta los retornos históricos para que reflejen la volatilidad actual del mercado. Se utiliza un modelo como EWMA para dar más peso a las observaciones recientes al calcular la volatilidad.

$$ \sigma_t^2 =λ \sigma_{t−1}^2 +(1-λ)r_{t−1}^2 $$


**Implementación:**

Cuando EWMA=True, para cada día t en la ventana de prueba:

* Se calculan las volatilidades históricas de los retornos de cada tasa en la ventana de 250 días.

* Se calcula la matriz de covarianza EWMA y de ella se extraen las volatilidades actuales para cada tasa: latest_sigma = sqrt(diag(S)) (Calcular hist_sigma como desviación estándar histórica de cada nodo).

* Los retornos de periodos de alta volatilidad pasada se amplifican si la volatilidad actual es alta, y viceversa.


In [ ]:
def portfolio_analysis(par_df, portfolio_def, tenors_months, lookback, alpha, ewma_lambda):
    T, M = par_df.shape
    tenors_years = np.array(tenors_months) / 12

    print("1. Realizando bootstrapping para todas las fechas...")
    zero_curves = np.array([bootstrap_zero_curve(row.values, tenors_months) for _, row in par_df.iterrows()])

    print("2. Pre-calculando flujos de caja del portafolio...")
    portfolio_cfs = get_portfolio_cashflows(portfolio_def)

    print("3. Calculando valor histórico del portafolio y P&L...")
    portfolio_values = np.array([valor_presente(portfolio_cfs, zc, tenors_years) for zc in zero_curves])
    actual_pnl = np.diff(portfolio_values)

    zero_log_returns = np.diff(np.log(zero_curves + 1e-9), axis=0)

    print("4. Iniciando simulación de VaR...")
    test_window_start = lookback + 1
    test_window_indices = np.arange(test_window_start, T)

    var_series_hs = np.zeros(len(test_window_indices))
    var_series_ewma = np.zeros(len(test_window_indices))

    for i, t in enumerate(test_window_indices):
        prev_zeros = zero_curves[t-1]
        lookback_idx = range(t - lookback - 1, t - 1)

        historical_log_returns = zero_log_returns[lookback_idx, :]

        # VaR Histórico
        scenario_curves_hs = prev_zeros * np.exp(historical_log_returns)
        scenario_pnl_hs = np.array([valor_presente(portfolio_cfs, sc, tenors_years) - portfolio_values[t-1] for sc in scenario_curves_hs])
        var_series_hs[i] = -np.percentile(scenario_pnl_hs, 100 * (1-alpha))

        # VaR Histórico con EWMA
        arithmetic_returns = np.exp(historical_log_returns) - 1
        S = np.cov(arithmetic_returns, rowvar=False)                  # Iniciar con cov histórica
        for r in arithmetic_returns:
            S = ewma_lambda * S + (1 - ewma_lambda) * np.outer(r, r)  # ope. los vectores

        latest_sigma = np.sqrt(np.diag(S) + 1e-9)
        hist_sigma = np.std(arithmetic_returns, axis=0, ddof=1) + 1e-9
        scaling_factor = latest_sigma / hist_sigma

        scaled_log_returns = historical_log_returns * scaling_factor
        scenario_curves_ewma = prev_zeros * np.exp(scaled_log_returns)
        scenario_pnl_ewma = np.array([valor_presente(portfolio_cfs, sc, tenors_years) - portfolio_values[t-1] for sc in scenario_curves_ewma])
        var_series_ewma[i] = -np.percentile(scenario_pnl_ewma, 100 * (1-alpha))

    print("5. Realizando backtesting...")
    actual_pnl_for_backtest = actual_pnl[test_window_indices - 1]

    exceptions_hs = actual_pnl_for_backtest < -var_series_hs
    exceptions_ewma = actual_pnl_for_backtest < -var_series_ewma

    n_obs_backtest = len(actual_pnl_for_backtest)
    p_kupiec_hs = kupiec_pof_test(np.sum(exceptions_hs), n_obs_backtest, alpha)
    p_kupiec_ewma = kupiec_pof_test(np.sum(exceptions_ewma), n_obs_backtest, alpha)

    backtest_results = {
        'hs': {'exceptions': np.sum(exceptions_hs), 'n_obs': n_obs_backtest, 'kupiec_p': p_kupiec_hs},
        'ewma': {'exceptions': np.sum(exceptions_ewma), 'n_obs': n_obs_backtest, 'kupiec_p': p_kupiec_ewma}
    }

    return {
        'portfolio_values': portfolio_values,
        'actual_pnl': actual_pnl,
        'var_series_hs': var_series_hs,
        'var_series_ewma': var_series_ewma,
        'test_window_indices': test_window_indices,
        'backtest': backtest_results
    }

In [ ]:
# Ejecutar la simulación y el backtesting
results = portfolio_analysis(bonds_yield, bonos, months, T, alpha, lambd)

In [ ]:
print("\nC. Resultados VaR")
print(f"   - VaR Histórico: ${results['var_series_hs'][-1]:,.2f}")
print(f"   - VaR Histórico con EWMA: ${results['var_series_ewma'][-1]:,.2f}")

print("\n RESULTADOS DEL BACKTESTING")
res_hs = results['backtest']['hs']
res_ewma = results['backtest']['ewma']
print("\n   Resultados para VaR Histórico Simple:")
print(f"     - Observaciones: {res_hs['n_obs']}")
print(f"     - Excepciones: {res_hs['exceptions']} (Tasa de fallo: {res_hs['exceptions']/res_hs['n_obs']:.2%})")
print(f"     - Test de Kupiec (POF): p-value = {res_hs['kupiec_p']:.4f} {'(Modelo OK)' if res_hs['kupiec_p'] > 0.05 else '(Modelo RECHAZADO)'}")

print("\n   Resultados para VaR Histórico Filtrado (EWMA):")
print(f"     - Observaciones: {res_ewma['n_obs']}")
print(f"     - Excepciones: {res_ewma['exceptions']} (Tasa de fallo: {res_ewma['exceptions']/res_ewma['n_obs']:.2%})")
print(f"     - Test de Kupiec (POF): p-value = {res_ewma['kupiec_p']:.4f} {'(Modelo OK)' if res_ewma['kupiec_p'] > 0.05 else '(Modelo RECHAZADO)'}")

**Gráfico del Valor del Portafolio**

Muestra la evolución del valor total de mercado de tu portafolio a lo largo del tiempo.

In [ ]:
portfolio_values = results['portfolio_values']
# Valor del Portafolio
plt.figure(figsize=(10, 3))
plt.plot(bonds_yield.index, portfolio_values, label='Valor del Portafolio')
plt.title('Valor de Mercado del Portafolio')
plt.xlabel('Fecha')
plt.ylabel('Valor de Mercado ($)')
plt.legend();

In [ ]:
# Backtesting de VaR
test_indices = results['test_window_indices']
test_dates = bonds_yield.index[test_indices]
pnl_for_plot = results['actual_pnl'][test_indices - 1]

plt.figure(figsize=(12, 6))
plt.plot(test_dates, pnl_for_plot, label='P&L Actual', color='gray', alpha=0.7, linewidth=1.5, zorder=1)
plt.plot(test_dates, -results['var_series_hs'], label=f'VaR {alpha*100:.0f}% Histórico', color='blue', linestyle='--', zorder=2)
plt.plot(test_dates, -results['var_series_ewma'], label=f'VaR {alpha*100:.0f}% EWMA', color='red', linestyle=':', zorder=3)

# Marcar las excepciones del modelo EWMA (suele ser más interesante)
exceptions_plot = pnl_for_plot < -results['var_series_ewma']
plt.scatter(test_dates[exceptions_plot], pnl_for_plot[exceptions_plot],
               color='red', marker='o', s=50, label='Excepciones (EWMA)', zorder=4, facecolors='none', edgecolors='k')

plt.title('Backtesting: P&L Actual vs. VaR Pronósticado')
plt.xlabel('Fecha')
plt.ylabel('Pérdida y Ganancia ($)')
plt.legend();

Mejoras y Extensiones Sugeridas:
---

**Construcción de la Curva**: El método de bootstrap es funcional, pero se puede mejorar significativamente para obtener una curva más suave y realista, lo que impactará directamente en la precisión de la valoración y cálculos de sensibilidades (medidas de riesgo).

Por ejemplo, los modelos Paramétricos  de Nelson-Siegel y/o Svensson. En lugar de la interpolación lineal del bootstrap, puedes ajustar la curva de tasas a un modelo paramétrico.

**Modelo GARCH**: EWMA es un caso particular de los modelos GARCH. Un modelo GARCH(1,1) es más flexible, ya que permite que la volatilidad revierta a una media a largo plazo, lo que a menudo es más realista que el supuesto de EWMA.

Notas finales:
---

**¿Por qué el ejemplo define solo tres bonos si tengo yields para muchos vencimientos?**

Hay varias razones, prácticas y didácticas, para usar un pequeño conjunto de bonos representativos en el ejemplo:

a) Suficiencia para exponer riesgo: Tres bonos “bien elegidos” (uno corto, uno medio, uno largo) capturan la mayor parte del riesgo de tasa de interés (duration y convexity) a lo largo de la curva. Para muchos propósitos ilustrativos —VaR, backtest— eso es suficiente.

b) Simplicidad / claridad didáctica: El ejemplo busca que se entienda el flujo: bootstrap → pricing → escenarios → VaR.

c) Representatividad y liquidez: En mercados reales se suelen usar bonos liquidos en ciertos vencimientos (ej.: 2y, 5y, 10y, 30y). Un subconjunto seleccionado cubre las partes líquidas de la curva.


**¿Cuándo conviene usar más bonos? (y cómo elegirlos)**

Usa más bonos cuando tu objetivo sea alguno de estos:

* Replicar exactamente la curva o replicar exposiciones a todos los tenores: necesitarás instrumentos que cubran toda la estructura.

* Replicar o seguir un índice (index-tracking).

* Hacer análisis de Cobertura(hedging) / immunización.

Criterios de selección práctica:

* Elige tenores representativos: por ejemplo 1y, 2y, 5y, 7y, 10y, 20y, 30y.

* Si buscas reproducir la curva: puedes resolver un problema de mínimos cuadrados para hallar cantidades q_j que minimicen la diferencia entre precios/DFs objetivo y los del portafolio replicante.


In [ ]:
print('Fin del ejemplo')